In [134]:
from torchvision import transforms
from torchvision.models import *
import matplotlib.pyplot as plt
import torchvision.models as models
from torchvision import transforms
import numpy as np
import torch
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
import torch.optim as optim
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import torchvision
import wandb
import os
import warnings
import torch.nn as nn
from torchsummary import summary
from torch.distributions import Normal, Independent, kl, MultivariateNormal
warnings.filterwarnings("ignore")

In [2]:
from my_VAE import *

ModuleNotFoundError: No module named 'my_VAE'

In [3]:
model = UNet(3, [64,128,256, 512, 1024], 10).to('cuda')

In [4]:
posterior = AxisAlignedConvGaussian(3, [64,128,256, 512, 1024], (256,256)).to('cuda')

In [5]:
input = torch.randn(1, 3, 256, 256).to('cuda')

In [6]:
out = model(input)

In [7]:
out = posterior(input)

In [8]:
summary(posterior, (3, 256, 256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]           1,792
       BatchNorm2d-2         [-1, 64, 256, 256]             128
              ReLU-3         [-1, 64, 256, 256]               0
            Conv2d-4         [-1, 64, 256, 256]          36,928
       BatchNorm2d-5         [-1, 64, 256, 256]             128
              ReLU-6         [-1, 64, 256, 256]               0
         AvgPool2d-7         [-1, 64, 128, 128]               0
            Conv2d-8        [-1, 128, 128, 128]          73,856
       BatchNorm2d-9        [-1, 128, 128, 128]             256
             ReLU-10        [-1, 128, 128, 128]               0
           Conv2d-11        [-1, 128, 128, 128]         147,584
      BatchNorm2d-12        [-1, 128, 128, 128]             256
             ReLU-13        [-1, 128, 128, 128]               0
        AvgPool2d-14          [-1, 128,

In [9]:
out.shape

torch.Size([1, 2])

In [10]:
posterior

AxisAlignedConvGaussian(
  (enc_layers): ModuleDict(
    (enc0_unetblock): Sequential(
      (enc0_conv1): Sequential(
        (enc0_conv1_conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (enc0_conv1_norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (enc0_conv1_relu): ReLU(inplace=True)
      )
      (enc0_conv2): Sequential(
        (enc0_conv2_conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (enc0_conv2_norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (enc0_conv2_relu): ReLU(inplace=True)
      )
    )
    (enc0_avgpooling): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (enc1_unetblock): Sequential(
      (enc1_conv1): Sequential(
        (enc1_conv1_conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (enc1_conv1_norm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [186]:
dist = Independent(MultivariateNormal(torch.randn(5,3), torch.eye(3,3)),1)

In [125]:
dist = Normal(torch.tensor([0.0]), torch.tensor([1.0]))

In [132]:
dist.sample()

tensor([[ 0.2646,  2.2858,  0.1620],
        [ 2.0168,  0.4195, -1.0985],
        [ 0.0867, -1.4262, -0.0457],
        [ 0.5519, -0.0608, -0.3402],
        [-0.7130,  0.6988, -0.1468]])

In [187]:
dim = 6
encoding = torch.randn(10, 3, 256, 256)

encoding = torch.mean(encoding, dim=2, keepdim=True)
encoding = torch.mean(encoding, dim=3, keepdim=True)

mu_log_sigma = nn.Conv2d(3, 2*dim, (1,1), stride=1)(encoding)
print(mu_log_sigma.shape)


torch.Size([10, 12, 1, 1])


In [188]:
#We squeeze the second dimension twice, since otherwise it won't work when batch size is equal to 1
mu_log_sigma = torch.squeeze(mu_log_sigma, dim=2)
mu_log_sigma = torch.squeeze(mu_log_sigma, dim=2)
print(mu_log_sigma.shape)

torch.Size([10, 12])


In [189]:
mu = mu_log_sigma[:,:dim]
log_sigma = mu_log_sigma[:,dim:]
print(mu.shape)
print(log_sigma.shape)

torch.Size([10, 6])
torch.Size([10, 6])


In [190]:
dist = Independent(Normal(loc = mu, scale = torch.exp(log_sigma)),1)

In [191]:
dist.sample()

tensor([[ 9.4490e-01,  1.0597e-01,  1.7293e-01,  1.3635e+00, -1.5120e-01,
         -1.4469e+00],
        [-2.6283e+00,  7.5443e-01,  7.2492e-01,  1.5548e+00, -9.1815e-01,
          7.9257e-01],
        [-1.1739e+00,  4.3533e-01,  5.1895e-01, -1.3991e-01,  1.3177e+00,
         -2.2122e-01],
        [ 3.1662e-01,  1.7385e-03,  3.0865e-01, -1.3608e+00, -3.4822e-01,
         -1.8382e+00],
        [-7.4337e-01,  2.0241e-01,  8.4742e-01,  7.1241e-01,  3.5360e-01,
         -7.2752e-01],
        [ 2.7363e-01, -3.1773e-01, -3.7578e-01, -3.5042e-01,  4.1933e-01,
         -9.9323e-02],
        [ 2.5041e-01, -8.0951e-01, -2.0993e-01,  8.0267e-01,  1.3261e+00,
          6.7523e-01],
        [-3.2769e+00, -6.4641e-01,  3.2656e-02, -2.4559e-02,  5.4121e-01,
         -6.7497e-01],
        [-3.1239e+00,  1.2777e+00,  7.4376e-01, -1.1216e+00,  1.6115e+00,
          4.2523e-01],
        [ 8.8957e-01,  9.1371e-01,  3.2939e-01, -1.7217e+00, -1.1753e-01,
         -3.8809e-01]])

In [200]:
def test():
    for i in range(16):
        yield i

In [204]:
j = 10
print(j)
for k in test():
    x =+ k
print(x)

10
15


In [209]:
from statistics import mean
c = [1,2,3,44,5]
mean(c)

11